In [1]:
from dados_de_mercado import Dados_de_mercado
import pandas as pd
from datetime import datetime, timedelta
import time
import os
import matplotlib.pyplot as plt

from fintz import Fintz

In [2]:
os.chdir("C:\\Users\\eduar\\dev\\github\\dados_fundamentalistas_fintz\\atualizado1q24")

In [3]:
precos = pd.read_parquet('historico_acoes.parquet')
ebit = pd.read_parquet('EBIT.parquet')
receita_liquida = pd.read_parquet('ReceitaLiquida.parquet')
custos = pd.read_parquet('Custos.parquet')
lucro_liquido = pd.read_parquet('LucroLiquido.parquet')
ativo_circulante = pd.read_parquet('AtivoCirculante.parquet')
ativo_nao_circulante = pd.read_parquet('AtivoNaoCirculante.parquet')
ativo_total = pd.read_parquet('AtivoTotal.parquet')
caixa = pd.read_parquet('CaixaEquivalentes.parquet')
divida_bruta = pd.read_parquet('DividaBruta.parquet')
divida_liquida = pd.read_parquet('DividaLiquida.parquet')
ebitda = pd.read_parquet('EBITDA.parquet')
passivo_circulante = pd.read_parquet('PassivoCirculante.parquet')
passivo_nao_circulante = pd.read_parquet('PassivoNaoCirculante.parquet')
passivo_total = pd.read_parquet('PassivoTotal.parquet')
patrimonio_liquido = pd.read_parquet('PatrimonioLiquido.parquet')
roe = pd.read_parquet('ROE.parquet')
p_l = pd.read_parquet('P_L.parquet')
p_vp = pd.read_parquet('P_VP.parquet')
dy = pd.read_parquet('DividendYield.parquet')
roic = pd.read_parquet('ROIC.parquet')
vpa = pd.read_parquet('VPA.parquet')
lpa = pd.read_parquet('LPA.parquet')
divida_bruta_patrimonio_liquido = pd.read_parquet('DividaBruta_PatrimonioLiquido.parquet')
divida_liquida_patrimonio_liquido = pd.read_parquet('DividaLiquida_PatrimonioLiquido.parquet')
divida_liquida_ebit = pd.read_parquet('DividaLiquida_EBIT.parquet')
divida_liquida_ebitda = pd.read_parquet('DividaLiquida_EBITDA.parquet')
valor_de_mercado = pd.read_parquet('ValorDeMercado.parquet')
ev = pd.read_parquet('EV.parquet')

In [4]:
os.chdir("C:\\Users\\eduar\\dev\\github\\radar_fundamentalista")

In [5]:
tickers = ['RRRP3', 'ABEV3','AZUL4','B3SA3','BBSE3','BBDC3','BBDC4','BBAS3','BRKM5','BRFS3','BPAC11','CMIG4','CIEL3','COGN3',
 'CPLE6','CSAN3','CMIN3','CVCB3','CYRE3','ELET3','EMBR3','EQTL3','GGBR4','GOAU4','NTCO3','HAPV3','IRBR3','ITSA4','ITUB4','JBSS3',
 'KLBN11','RENT3','LREN3','LWSA3','MGLU3','MRFG3','BEEF3','MRVE3','PCAR3','PETR4','PRIO3','PETZ3','SBSP3','SANB11','CSNA3','SLCE3',
 'SUZB3','TAEE11','VIVT3','TIMS3','UGPA3','USIM5','VALE3','WEGE3','YDUQ3']

In [6]:
# Tickers que estão no radar e não estão entre as principais ações da bolsa:
tickers_radar = ['CMIG3','TAEE3','TAEE4','CPLE3','CSMG3','SAPR11','SAPR3','SAPR4','BRSR3','BRSR5','BRSR6','ITSA3','SANB3','SANB4','ITUB3',
'PSSA3','WIZC3','ENAT3']

In [7]:
for ticker in tickers_radar:
    tickers.append(ticker)

In [118]:
ticker = 'VALE3'

In [119]:
anos = list(range(14,25,1))
anos

[14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]

In [120]:
# evolucao de caixa
dados_caixa = caixa[caixa['ticker']==ticker]

In [121]:
dados_caixa_list = []
for ano in anos:
    dado = dados_caixa[dados_caixa['data']==f'20{ano}-01-01']['valor'].values[0]
    dados_caixa_list.append(dado)

ultimo_caixa = dados_caixa_list[-1]
caixa_anterior = ultimo_caixa
score = 0
for i in range(1,6):
    caixa_ano = dados_caixa_list[-(i+1)]

    if ultimo_caixa > caixa_ano:
        score += 0.5

    if caixa_anterior > caixa_ano:
        score += 0.5

    if ultimo_caixa < caixa_ano*0.5 or caixa_anterior < caixa_ano*0.67:
        score -= 1

    print(f'2024: {ultimo_caixa}, anterior: {caixa_anterior}, ano:{caixa_ano}, score: {score}')
    caixa_anterior = caixa_ano

score /= 5
print(score)

2024: 19864000000.0, anterior: 19864000000.0, ano:28015000000.0, score: 0
2024: 19864000000.0, anterior: 28015000000.0, ano:59057151000.0, score: -1
2024: 19864000000.0, anterior: 59057151000.0, ano:49889394000.0, score: -1.5
2024: 19864000000.0, anterior: 49889394000.0, ano:35644435000.0, score: -1.0
2024: 19864000000.0, anterior: 35644435000.0, ano:24424373000.0, score: -0.5
-0.1


In [122]:
# evolução divida longo prazo
dados_divida_longa = passivo_nao_circulante[passivo_nao_circulante['ticker']==ticker]

In [123]:
dados_divida_longa_list = []
for ano in anos:
    dado = dados_divida_longa[dados_divida_longa['data']==f'20{ano}-01-01']['valor'].values[0]
    dados_divida_longa_list.append(dado)

ultima_divida = dados_divida_longa_list[-1]
ultima_divida = round(ultima_divida,2)
divida_anterior = ultima_divida
score = 0
for i in range(1,6):
    divida_ano = dados_divida_longa_list[-(i+1)]
    divida_ano = round(divida_ano,2)

    if ultima_divida < divida_ano:
        score += 0.5

    if divida_anterior < divida_ano:
        score += 0.5

    if ultima_divida > divida_ano*2 or divida_anterior > divida_ano*2:
        score -= 1

    print(f'2024: {ultima_divida}, anterior: {divida_anterior}, ano:{divida_ano}, score: {score}')
    divida_anterior = divida_ano

score /= 5
print(score)

score /= 5

2024: 179632000000.0, anterior: 179632000000.0, ano:178094000000.0, score: 0
2024: 179632000000.0, anterior: 178094000000.0, ano:199664424000.0, score: 1.0
2024: 179632000000.0, anterior: 199664424000.0, ano:203518099000.0, score: 2.0
2024: 179632000000.0, anterior: 203518099000.0, ano:157385547000.0, score: 2.0
2024: 179632000000.0, anterior: 157385547000.0, ano:138866793000.0, score: 2.0
0.4


In [124]:
# Dados número de ações:
dados_patrimonio_liquido = patrimonio_liquido[patrimonio_liquido['ticker']==ticker]

In [125]:
dados_vpa = vpa[vpa['ticker']==ticker]

In [138]:
numero_acoes_list = []
for ano in anos:
    dado_pl = dados_patrimonio_liquido[dados_patrimonio_liquido['data']==f'20{ano}-01-01']['valor'].values[0]
    dado_vpa = dados_vpa[dados_vpa['data']==f'20{ano}-01-01']['valor'].values[0]
    num_acoes = dado_pl/dado_vpa
    num_acoes = num_acoes/1000000
    num_acoes = round(num_acoes,-1)
    numero_acoes_list.append(num_acoes)

ultimo_num = numero_acoes_list[-1]
score = 0
ano = 2024
for i in range(1,6):
    num_ano = numero_acoes_list[-(i+1)]
    if ultimo_num < num_ano*0.99:
        score += 1
    elif ultimo_num > num_ano*1.01:
        score -= 1

    print(f'{ano}: {ultimo_num}, {ano-1}: {num_ano}, score: {score}')
    ultimo_num = num_ano
    ano -= 1

score /= 5
print(score)

2024: 4540.0, 2023: 4780.0, score: 1
2023: 4780.0, 2022: 5130.0, score: 2
2022: 5130.0, 2021: 5280.0, score: 3
2021: 5280.0, 2020: 5280.0, score: 3
2020: 5280.0, 2019: 5280.0, score: 3
0.6


In [127]:
# Dados dívida bruta/patrimonio liqdo
dados_divida_pl = divida_bruta_patrimonio_liquido[divida_bruta_patrimonio_liquido['ticker']==ticker]

In [128]:
dados_divida_pl_list = []
for ano in anos:
    dado = dados_divida_pl[dados_divida_pl['data']==f'20{ano}-01-01']['valor'].values[0]
    dados_divida_pl_list.append(dado)

ultima_divida = dados_divida_pl_list[-1]
ultima_divida = round(ultima_divida,2)
divida_anterior = ultima_divida
score = 0
for i in range(1,6):
    divida_ano = dados_divida_pl_list[-(i+1)]
    divida_ano = round(divida_ano,2)

    if ultima_divida < divida_ano:
        score += 0.5

    if divida_anterior < divida_ano:
        score += 0.5

    if ultima_divida > divida_ano*2 or divida_anterior > divida_ano*2:
        score -= 1

    print(f'2024: {ultima_divida}, anterior: {divida_anterior}, ano:{divida_ano}, score: {score}')
    divida_anterior = divida_ano

score /= 5
print(score)

2024: 0.36, anterior: 0.36, ano:0.35, score: 0
2024: 0.36, anterior: 0.35, ano:0.4, score: 1.0
2024: 0.36, anterior: 0.4, ano:0.45, score: 2.0
2024: 0.36, anterior: 0.45, ano:0.39, score: 2.5
2024: 0.36, anterior: 0.39, ano:0.4, score: 3.5
0.7


In [129]:
# Dados Lucro liquido e custos
dados_lucro_liquido = lucro_liquido[lucro_liquido['ticker']==ticker]
dados_custos = custos[custos['ticker']==ticker]

In [130]:
dados_lucro_list = []

for ano in anos:
    dado = dados_lucro_liquido[dados_lucro_liquido['data']==f'20{ano}-01-01']['valor'].values[0]
    dados_lucro_list.append(dado)

ultimo_lucro = dados_lucro_list[-1]
lucro_anterior = ultimo_lucro
score = 0
for i in range(1,6):
    lucro_ano = dados_lucro_list[-(i+1)]

    if ultimo_lucro > lucro_ano:
        score += 0.5

    if lucro_anterior > lucro_ano:
        score += 0.5

    if ultimo_lucro < lucro_ano*0.67 or lucro_anterior < lucro_ano*0.67:
        score -= 1

    print(f'2024: {ultimo_lucro}, anterior: {lucro_anterior}, ano:{lucro_ano}, score: {score}')
    lucro_anterior = lucro_ano

score /= 5
print(score)

2024: 13966000000.0, anterior: 13966000000.0, ano:23228000000.0, score: -1
2024: 13966000000.0, anterior: 23228000000.0, ano:20574181000.0, score: -0.5
2024: 13966000000.0, anterior: 20574181000.0, ano:15072230000.0, score: 0.0
2024: 13966000000.0, anterior: 15072230000.0, ano:6461278000.0, score: 1.0
2024: 13966000000.0, anterior: 6461278000.0, ano:5607654000.0, score: 2.0
0.4


In [131]:
dados_custos_list = []

for ano in anos:
    dado = dados_custos[dados_custos['data']==f'20{ano}-01-01']['valor'].values[0]
    dados_custos_list.append(dado)

ultimo_custo = dados_custos_list[-1]
custo_anterior = 1
score = 0
for i in range(1,6):
    custo_ano = dados_custos_list[-(i+1)]

    if ultimo_custo > custo_ano:
        score += 0.5

    if custo_anterior > custo_ano:
        score += 0.5

    if ultimo_custo < custo_ano*1.5 or custo_anterior < custo_ano*1.5:
        score -= 1

    print(f'2024: {ultimo_custo}, anterior: {custo_anterior}, ano:{custo_ano}, score: {score}')
    custo_anterior = custo_ano

score /= 5
print(score)

2024: -30851000000.0, anterior: 1, ano:-33043000000.0, score: 1.0
2024: -30851000000.0, anterior: -33043000000.0, ano:-30533593000.0, score: 1.0
2024: -30851000000.0, anterior: -30533593000.0, ano:-25893133000.0, score: 1.0
2024: -30851000000.0, anterior: -25893133000.0, ano:-22627536000.0, score: 1.0
2024: -30851000000.0, anterior: -22627536000.0, ano:-22827339000.0, score: 1.5
0.3


In [132]:
# Dados dividendo
dados_preco = precos[precos['ticker']==ticker]
dados_dy = dy[dy['ticker']==ticker]
dados_lpa = lpa[lpa['ticker']==ticker]

In [133]:
dividendos_list = []
payout_list = []
for ano in anos:

    data_inicio_ano = pd.to_datetime(f'20{ano}-01-01')
    primeira_data_apos_inicio_ano = dados_preco.loc[dados_preco['data'] > data_inicio_ano, 'data'].min()
    dado_preco = dados_preco.loc[dados_preco['data'] == primeira_data_apos_inicio_ano, 'preco_abertura'].values[0]
    dado_dy = dados_dy.loc[dados_dy['data'] == primeira_data_apos_inicio_ano, 'valor'].values[0]
    dado_lpa = dados_lpa.loc[dados_lpa['data'] == primeira_data_apos_inicio_ano, 'valor'].values[0]

    dividendos = dado_preco*dado_dy
    dividendos_list.append(dividendos)

    payout = dividendos/dado_lpa
    payout_list.append(payout)
    
ultimo_div = dividendos_list[-1]
div_anterior = 99999990
score = 0
for i in range(1,6):
    div_ano = dividendos_list[-(i+1)]
    payout_ano = payout_list[-(i+1)]

    if ultimo_div > div_ano:
        score += 0.5

    if div_anterior > div_ano:
        score += 0.5

    if ultimo_div < div_ano*0.5 or div_anterior < div_ano*0.5:
        score -= 1

    # if payout_ano > 0.8:
    #     score += 0.5
    # elif payout_ano > 0.6:
    #     score += 0.3
    # elif payout_ano > 0.4:
    #     score += 0.1

    if payout_ano > 0:
        score *= payout_ano
    else:
        score *= 0.1

    print(f'2024: {ultimo_div}, anterior: {div_anterior}, ano:{div_ano}, payout: {payout_ano}, score: {score}')
    div_anterior = div_ano

score /= 5
score = round(score,4)
print(score)

2024: 6.121164032892927, anterior: 99999990, ano:7.52225207682953, payout: 0.33680943404622643, score: 0.16840471702311322
2024: 6.121164032892927, anterior: 7.52225207682953, ano:14.836358734294873, payout: 0.7957922243279929, score: -0.6617770599808435
2024: 6.121164032892927, anterior: 14.836358734294873, ano:2.3519689791384213, payout: 0.8028544908670768, score: 0.2715438063086458
2024: 6.121164032892927, anterior: 2.3519689791384213, ano:1.405252873893245, payout: 0.5221944727844097, score: 0.6639931475576247
2024: 6.121164032892927, anterior: 1.405252873893245, ano:1.9272529699177916, payout: 0.7431772882934994, score: 0.8650532709940907
0.173


In [134]:
# Divida liqu / pl
dados_div_liq_pl = divida_liquida_patrimonio_liquido[divida_liquida_patrimonio_liquido['ticker']==ticker]

In [135]:
dados_divida_liq_pl_list = []
for ano in anos:
    dado = dados_div_liq_pl[dados_div_liq_pl['data']==f'20{ano}-01-01']['valor'].values[0]
    dados_divida_liq_pl_list.append(dado)

ultima_divida = dados_divida_liq_pl_list[-1]
ultima_divida = round(ultima_divida,2)
divida_anterior = 100
score = 0
for i in range(1,6):
    divida_ano = dados_divida_liq_pl_list[-(i+1)]
    divida_ano = round(divida_ano,2)

    if ultima_divida < divida_ano:
        score += 0.5

    if divida_anterior < divida_ano:
        score += 0.5

    if ultima_divida > divida_ano*2 or divida_anterior > divida_ano*2:
        score -= 1

    print(f'2024: {ultima_divida}, anterior: {divida_anterior}, ano:{divida_ano}, score: {score}')
    divida_anterior = divida_ano

score /= 5
print(score)

2024: 0.25, anterior: 100, ano:0.2, score: -1
2024: 0.25, anterior: 0.2, ano:0.08, score: -2
2024: 0.25, anterior: 0.08, ano:0.19, score: -1.5
2024: 0.25, anterior: 0.19, ano:0.19, score: -1.5
2024: 0.25, anterior: 0.19, ano:0.26, score: -0.5
-0.1


In [136]:
# Divida liqu / ebitda
dados_div_ebitda = divida_liquida_ebitda[divida_liquida_ebitda['ticker']==ticker]

In [137]:
dados_div_ebitda_list = []
for ano in anos:
    dado = dados_div_ebitda[dados_div_ebitda['data']==f'20{ano}-01-01']['valor'].values[0]
    dados_div_ebitda_list.append(dado)

ultima_divida = dados_div_ebitda_list[-1]
ultima_divida = round(ultima_divida,2)
divida_anterior = 100
score = 0
for i in range(1,6):
    divida_ano = dados_div_ebitda_list[-(i+1)]
    divida_ano = round(divida_ano,2)

    if ultima_divida < divida_ano:
        score += 0.5

    if divida_anterior < divida_ano:
        score += 0.5

    if ultima_divida > divida_ano*2 or divida_anterior > divida_ano*2:
        score -= 1

    print(f'2024: {ultima_divida}, anterior: {divida_anterior}, ano:{divida_ano}, score: {score}')
    divida_anterior = divida_ano

score /= 5
print(score)

2024: 0.63, anterior: 100, ano:0.35, score: -1
2024: 0.63, anterior: 0.35, ano:0.11, score: -2
2024: 0.63, anterior: 0.11, ano:0.66, score: -1.0
2024: 0.63, anterior: 0.66, ano:0.86, score: 0.0
2024: 0.63, anterior: 0.86, ano:0.83, score: 0.5
0.1
